# p-adic distance with fixed-point precision arithmetics

* The input is a real value represented by float x
* Since x has finite floating point, it is a rational number
* Get fractional representation of x as a rational number, (p, q). ( x = p / q )
    * p is actually x * 1e12 (the max decimals)
    * q is equal to 1e12 (the max decimals)
    * then, find the gcd and divide them by the gcd.
* Calculate the p-adic valucations v(p) and v(q)
* Subtract v(p) - v(q) to get the p-adic absolute value of x.

In [1]:
import Data.Fixed

## Convert real (float, double) to fixed-point 

In [2]:
realToFixedE12 :: Real x => x -> Fixed E12
realToFixedE12 x = realToFrac x :: Fixed E12

In [3]:
intPart :: HasResolution a => Fixed a -> Int
intPart x = div' x 1

### test

In [4]:
realToFixedE12 10

10.000000000000

In [5]:
realToFixedE12 3.1415

3.141500000000

## Get fractional representation

In [6]:
let x = realToFixedE12 3.14
x

3.140000000000

In [7]:
let maxDec = realToFixedE12 1e12
maxDec

1000000000000.000000000000

In [8]:
x * maxDec

3140000000000.000000000000

In [9]:
intPart (x * maxDec)

3140000000000

In [10]:
realToFrac 1e12

1.0e12

In [11]:
round 1e12

1000000000000

In [12]:
x * intPart 1e12

: 

In [13]:
fracRep :: Fixed E12 -> (Int, Int)
fracRep x = (p', q')
    where
        maxDecimal = realToFixedE12 1e12
        p = intPart (x * maxDecimal)
        q = intPart maxDecimal
        gcd_val = gcd p q        
        p' = p `div` gcd_val
        q' = q `div` gcd_val

## p-adic distance

### p-adic valuation

In [14]:
padicValuation :: Int -> Int -> Int
padicValuation prime num
    | num `mod` prime /=0 = 0 -- if prime does not divide num
    | otherwise = 1 + padicValuation prime (num `div` prime)

In [15]:
padicValuation 2 6

1

In [16]:
padicValuation 2 12

2

In [17]:
padicValuation 2 2

1

In [18]:
let pq = fracRep 3.14
pq

(157,50)

In [19]:
uncurry (+) (fracRep 3.14)

207

In [20]:
import Control.Arrow ((***))

In [21]:
padicAbs :: Fractional a => Int -> Fixed E12 -> a
padicAbs prime x = fromIntegral prime ^^ (-val_diff)
    where
        pq = fracRep x -- (Int, Int)
        valuationFunc = padicValuation prime
        val_pair = (valuationFunc *** valuationFunc ) pq -- (Int, Int)
        val_diff = uncurry (-) val_pair -- Int
        

In [22]:
padicAbs 2 2

0.5

In [23]:
padicValuation 2 2

1

In [42]:
padicDist :: Fractional a => Int -> Fixed E12 -> Fixed E12 -> a
padicDist prime x y = padicAbs prime (x - y)

In [43]:
padicDist 2 3 2

1.0

In [44]:
padicDist 2 4 2

0.5

In [45]:
padicDist 2 4 0

0.25

In [46]:
padicDist 5 75 0

4.0e-2

# Experiments

The following fails because x is not a fraction.

In [24]:
convertToFixedE12 :: Real x => x -> Fixed E12
convertToFixedE12 x = x :: Fixed E12


: 

In [25]:
:info realToFrac 

In [26]:
(realToFixedE12 . realToFixedE12) 3.14

3.140000000000

In [27]:
3.14 & (+1)

: 

In [28]:
import Data.Function ((&))

In [29]:
3.14 & (+1)

4.140000000000001

In [30]:
3.14 & realToFixedE12 & (+1)

4.140000000000

In [31]:
1.1 ::  Fixed E12

1.100000000000

In [32]:
(realToFrac 1.1) :: Fixed E12

1.100000000000

In [33]:
1.1 :: HasResolution E12

: 

In [34]:
fracPart :: HasResolution a=> Fixed a->Fixed a
fracPart x = mod' x 1

In [35]:
intPart :: HasResolution a => Fixed a -> Int
intPart x = div' x 1

In [36]:
intPart (3.14 :: Fixed E12)

3

In [37]:
3.14 & realToFixedE12 & (*1e12) & intPart & (gcd (intPart (1e12 :: Fixed E12)))

Line 1: Redundant bracket
Found:
3.14 & realToFixedE12 & (* 1e12) & intPart
  & (gcd (intPart (1e12 :: Fixed E12)))
Why not:
3.14 & realToFixedE12 & (* 1e12) & intPart
  & gcd (intPart (1e12 :: Fixed E12))

20000000000

In [38]:
intPart 1e3

: 

In [39]:
intPart :: HasResolution a => Fixed a -> Fixed E12
intPart x = (x `mod'` 1) & realToFixedE12

In [40]:
realToFrac 1.1

1.1